<a href="https://colab.research.google.com/github/KharchenkoAnastasia/sql/blob/main/SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pandasql


  Preparing metadata (setup.py) ... done


In [2]:
import pandas as pd
import pandasql as psql


In [3]:
df = pd.read_csv('/content/ds_salaries.csv')
df.columns

Index(['work_year', 'experience_level', 'employment_type', 'job_title',
       'salary', 'salary_currency', 'salary_in_usd', 'employee_residence',
       'remote_ratio', 'company_location', 'company_size'],
      dtype='object')

In [4]:
df.columns=["year","exp_level","emp_type","job_title", "salary", "salary_curr","salary_in_usd","emp_location", "remote_ratio", "company_location", "company_size"]

In [5]:
query = """
SELECT
    year
    , job_title
    , remote_ratio
    , salary_in_usd
FROM df
WHERE
    remote_ratio == 100
ORDER BY 4 DESC
----LIMIT 5
"""
result = psql.sqldf(query)
print(result)


      year                 job_title  remote_ratio  salary_in_usd
0     2021  Principal Data Scientist           100         416000
1     2020            Data Scientist           100         412000
2     2022       Data Analytics Lead           100         405000
3     2022    Applied Data Scientist           100         380000
4     2023            Data Architect           100         376080
...    ...                       ...           ...            ...
1638  2022           BI Data Analyst           100           6270
1639  2020      Product Data Analyst           100           6072
1640  2022              Data Analyst           100           5723
1641  2021            Data Scientist           100           5679
1642  2022              NLP Engineer           100           5132

[1643 rows x 4 columns]


#**Lesson 5 Пишемо SELECT, додаємо функції, агрегації та CASE WHEN**





In [6]:
query = """
SELECT
    exp_level
  --  , salary_in_usd
   -- , salary_in_usd * 38
    , CASE
      WHEN exp_level = 'SE'
      THEN "Senior"
      WHEN exp_level = 'MI'
      THEN "Middle"
      ELSE 'Othere' END AS FULL_level

FROM df
"""
result = psql.sqldf(query)
print(result)

     exp_level FULL_level
0           SE     Senior
1           MI     Middle
2           MI     Middle
3           SE     Senior
4           SE     Senior
...        ...        ...
3750        SE     Senior
3751        MI     Middle
3752        EN     Othere
3753        EN     Othere
3754        SE     Senior

[3755 rows x 2 columns]


 1. Вивести кількість унікальних значень для кожної колонки,
що містить текстові значення.

In [7]:
query = """
SELECT
    COUNT(DISTINCT job_title)
FROM df
"""
result = psql.sqldf(query)
print(result)

   COUNT(DISTINCT job_title)
0                         93


2. Вивести унікальні значення для кожної колонки, що містить текстові значення

In [8]:
query = """
SELECT DISTINCT job_title
FROM df
"""
result = psql.sqldf(query)
print(result)

                   job_title
0   Principal Data Scientist
1                ML Engineer
2             Data Scientist
3          Applied Scientist
4               Data Analyst
..                       ...
88  Head of Machine Learning
89    Principal Data Analyst
90   Principal Data Engineer
91      Staff Data Scientist
92      Finance Data Analyst

[93 rows x 1 columns]


3. Вивести середню, мінімальну та максимальну з/п (salary_in_usd) для кожного року (окремими запитами, в кожному з яких впроваджено фільтр відповідного року)

In [9]:
query = """
SELECT
    ROUND(AVG(salary_in_usd),0) AS avg_salary
    , MAX(salary_in_usd)        AS max_salary
    , MIN(salary_in_usd)        AS min_salary
FROM df
WHERE year='2021';
"""
result = psql.sqldf(query)
print(result)

   avg_salary  max_salary  min_salary
0     94087.0      423000        5409


4. Вивести середню з/п (salary_in_usd) для 2023 року по кожному рівню досвіду працівників (окремими запитами, в кожному з яких впроваджено фільтр року та досвіду).

In [10]:
query = """
SELECT
    exp_level
    , ROUND (AVG(salary_in_usd),0) AS avg_salary

FROM df
WHERE year = '2023'
GROUP BY exp_level;
"""
result = psql.sqldf(query)
print(result)

  exp_level  avg_salary
0        EN     95284.0
1        EX    203706.0
2        MI    116298.0
3        SE    159569.0


In [11]:
query = """
SELECT ROUND(AVG(salary_in_usd),0)	AS avg_salary
FROM df
WHERE year = 2023

"""
result = psql.sqldf(query)
print(result)

   avg_salary
0    149046.0


In [12]:
(95284.0+203706.0+ 116298.0+159569.0)/4

143714.25

In [13]:
query = """
SELECT
    exp_level
    , ROUND(AVG(salary_in_usd), 0) AS avg_salary
    , COUNT(*) AS record_count
    , SUM(salary_in_usd) AS total_salary
FROM df
WHERE year = '2023'
GROUP BY exp_level;
"""

result = psql.sqldf(query)
print(result)

  exp_level  avg_salary  record_count  total_salary
0        EN     95284.0           118      11243508
1        EX    203706.0            60      12222341
2        MI    116298.0           320      37215231
3        SE    159569.0          1287     205365211


5. Вивести 5 найвищих заробітних плат в 2023 році для представників спеціальності ML Engineer. Заробітні плати перевести в гривні.

In [14]:
query = """
SELECT
    salary_in_usd * 38 as salary_in_uah
FROM df
WHERE year = '2023' AND "job_title" = "ML Engineer"
ORDER BY salary_in_usd DESC
LIMIT 5;
"""

result = psql.sqldf(query)
print(result)

   salary_in_uah
0       10984888
1        9880000
2        8895800
3        8360000
4        7733000


6. Вивести Унікальні значення колонки remote_ratio, формат даних має бути дробовим з двома знаками після коми, приклад: значення 50 має відображатись в форматі 0.50

In [15]:
query = """
SELECT
    DISTINCT ROUND((remote_ratio)/100.0,2) AS remote_ratio_round
FROM df
"""

result = psql.sqldf(query)
print(result)

   remote_ratio_round
0                 1.0
1                 0.0
2                 0.5


7. Вивести дані таблиці, додавши колонку 'exp_level_full' з повною назвою рівнів досвіду працівників відповідно до колонки exp_level. Визначення: Entry-level (EN), Mid-level (MI), Senior-level (SE), Executive-level (EX)

In [16]:
query = """
SELECT
    exp_level
    , CASE
       WHEN exp_level = 'EN' THEN 'Entry-level'
       WHEN exp_level = 'MI' THEN 'Mid-level'
       WHEN exp_level = 'SE' THEN 'Senior-level '
       WHEN 'EX' THEN 'Executive-level'
       ELSE 'Othere' END AS exp_level_full
FROM df
"""

result = psql.sqldf(query)
print(result)

     exp_level exp_level_full
0           SE  Senior-level 
1           MI      Mid-level
2           MI      Mid-level
3           SE  Senior-level 
4           SE  Senior-level 
...        ...            ...
3750        SE  Senior-level 
3751        MI      Mid-level
3752        EN    Entry-level
3753        EN    Entry-level
3754        SE  Senior-level 

[3755 rows x 2 columns]


8. Додатки колонку "salary_category', яка буде відображати різні категорії заробітних плат відповідно до їх значення в колонці 'salary_in_usd'. Визначення: з/п менша за 20 000 - Категорія 1, з/п менша за 50 000 - Категорія 2, з/п менша за 100 000 - Категорія 3, з/п більша за 100 000 - Категорія 4

In [17]:
query = """
SELECT *
       , CASE
          WHEN salary_in_usd <= 20000 THEN "category 1"
          WHEN salary_in_usd <= 50000 THEN "category 2"
          WHEN salary_in_usd <= 100000 THEN "category 3"
          ELSE "category 4" END AS salary_category
FROM df
LIMIT 100;
"""

result = psql.sqldf(query)
print(result)

    year exp_level emp_type                  job_title  salary salary_curr  \
0   2023        SE       FT   Principal Data Scientist   80000         EUR   
1   2023        MI       CT                ML Engineer   30000         USD   
2   2023        MI       CT                ML Engineer   25500         USD   
3   2023        SE       FT             Data Scientist  175000         USD   
4   2023        SE       FT             Data Scientist  120000         USD   
..   ...       ...      ...                        ...     ...         ...   
95  2023        EN       FT  Machine Learning Engineer  163196         USD   
96  2023        EN       FT  Machine Learning Engineer  145885         USD   
97  2023        SE       FT              Data Engineer  217000         USD   
98  2023        SE       FT              Data Engineer  185000         USD   
99  2023        SE       FT               Data Analyst  202800         USD   

    salary_in_usd emp_location  remote_ratio company_location c

9. Дослідити всі колонки на наявність відсутніх значень, порівнявши кількість рядків таблиці з кількістю значень відповідної колонки

In [18]:
query = """
SELECT
      COUNT(*)      AS total_numb
      , COUNT(job_title) AS count_categ
      , COUNT(*) - COUNT(job_title) AS null_nmb
FROM df
"""

result = psql.sqldf(query)
print(result)

   total_numb  count_categ  null_nmb
0        3755         3755         0


10. Порахувати кількість працівників в таблиці, які в 2023 році працюють на компанії розміру "М" і отримують з/п вищу за $100 000

In [19]:
query = """
SELECT COUNT(*)
FROM salaries
WHERE 1=1
	AND year = '2023'
	AND company_size = 'M'
	AND salary_in_usd >= 100000;
"""

#result = psql.sqldf(query)
#print(result)

# **Lessons 6 ТОП 10 задач SQL для СПІВБЕСІДИ - легкий рівень (sql практикум для дата-аналітиків)**

In [20]:
df.columns

Index(['year', 'exp_level', 'emp_type', 'job_title', 'salary', 'salary_curr',
       'salary_in_usd', 'emp_location', 'remote_ratio', 'company_location',
       'company_size'],
      dtype='object')

1. Вивести з/п спеціалістів ML Engineer в 2023 році, додати сортування за зростанням з/п.

In [21]:
query = """
SELECT
    salary_in_usd AS salary
    , year
    , job_title
FROM df
WHERE
      year = 2023
      AND job_title = 'ML Engineer'
ORDER BY salary_in_usd ASC
LIMIT 10;
"""

result = psql.sqldf(query)
print(result)

   salary  year    job_title
0   25500  2023  ML Engineer
1   30000  2023  ML Engineer
2  110000  2023  ML Engineer
3  135000  2023  ML Engineer
4  147000  2023  ML Engineer
5  150000  2023  ML Engineer
6  160000  2023  ML Engineer
7  200000  2023  ML Engineer
8  202353  2023  ML Engineer
9  203500  2023  ML Engineer


2. Назвати країну (company_location),
в якій зафіксована найменша з/п спеціаліста в сфері Data Scientist в 2023 році

In [22]:
query = """
SELECT
      company_location
      , salary_in_usd
      , job_title
      , year
FROM df
WHERE
      year = 2023
      AND job_title = "Data Scientist"
ORDER BY 2 ASC
LIMIT 1;
"""

result = psql.sqldf(query)
print(result)

  company_location  salary_in_usd       job_title  year
0               IN           9727  Data Scientist  2023


3. Вивести топ 5 з/п серед усіх спеціалістів, які працюють повністю віддалено (remote_ratio = 100)

In [23]:
query = """
SELECT
      job_title
      , salary_in_usd
FROM df
WHERE remote_ratio = 100
ORDER BY 2 DESC
LIMIT 5;
"""

result = psql.sqldf(query)
print(result)

                  job_title  salary_in_usd
0  Principal Data Scientist         416000
1            Data Scientist         412000
2       Data Analytics Lead         405000
3    Applied Data Scientist         380000
4            Data Architect         376080


5. Вивести кількість унікальних значень колонки

In [24]:
query = """
SELECT COUNT(DISTINCT company_location)
FROM df
"""

result = psql.sqldf(query)
print(result)

   COUNT(DISTINCT company_location)
0                                72


6. Вивести середню, мінімальну та максимальну з/п (salary_in_usd) для 2023 року.

In [25]:
query = """
SELECT
    ROUND(AVG(salary_in_usd),2) AS AVG
    , MIN(salary_in_usd) AS MIN
    , MAX(salary_in_usd) AS MAX
FROM df
WHERE year = 2023
"""

result = psql.sqldf(query)
print(result)

         AVG   MIN     MAX
0  149045.54  7000  423834


7. Вивести 5 найвищих заробітних плат в 2023 році для представників спеціальності
ML Engineer. Заробітні плати перевести в гривні.

In [26]:
query = """
SELECT
     salary_in_usd * 38
     , job_title
FROM df
WHERE
      year = 2023
      AND job_title = 'ML Engineer'
ORDER BY 1 DESC
LIMIT 5
"""

result = psql.sqldf(query)
print(result)

   salary_in_usd * 38    job_title
0            10984888  ML Engineer
1             9880000  ML Engineer
2             8895800  ML Engineer
3             8360000  ML Engineer
4             7733000  ML Engineer


# **Lessons 7. Фільтри даних**

In [27]:
query = """
SELECT DISTINCT(year)
FROM df
WHERE 1=1
  AND YEAR
LIMIT 20;
"""

result = psql.sqldf(query)
print(result)

   year
0  2023
1  2022
2  2020
3  2021


 Вивести всіх співробітників, які в 2023 отримували з/п більшу за $300тис.

In [28]:
query = """
SELECT
      job_title
      , salary_in_usd
FROM df
WHERE
      salary_in_usd >= 300000
      AND year = 2023
"""

result = psql.sqldf(query)
print(result)

                    job_title  salary_in_usd
0    Computer Vision Engineer         342810
1           Applied Scientist         309400
2   Machine Learning Engineer         342300
3   Machine Learning Engineer         318300
4           Applied Scientist         309400
5   Machine Learning Engineer         300000
6                Head of Data         329500
7   Machine Learning Engineer         304000
8    Director of Data Science         353200
9              Data Scientist         317070
10               AI Scientist         423834
11             Data Architect         376080
12         Research Scientist         340000
13              Data Engineer         310000
14              Data Engineer         310000
15             Data Scientist         300240
16             Data Scientist         300240
17             Data Scientist         370000
18  Machine Learning Engineer         323300
19              Data Engineer         310000
20  Machine Learning Engineer         318300
21        

2.Вивести всіх співробітників, які в 2023 отримували з/п більшу за $300тис. та не працювали в великих компаніях.

In [29]:
query = """
SELECT
      job_title
      , salary_in_usd
FROM df
WHERE
      salary_in_usd >= 300000
      AND year = 2023
"""

result = psql.sqldf(query)
print(result)

                    job_title  salary_in_usd
0    Computer Vision Engineer         342810
1           Applied Scientist         309400
2   Machine Learning Engineer         342300
3   Machine Learning Engineer         318300
4           Applied Scientist         309400
5   Machine Learning Engineer         300000
6                Head of Data         329500
7   Machine Learning Engineer         304000
8    Director of Data Science         353200
9              Data Scientist         317070
10               AI Scientist         423834
11             Data Architect         376080
12         Research Scientist         340000
13              Data Engineer         310000
14              Data Engineer         310000
15             Data Scientist         300240
16             Data Scientist         300240
17             Data Scientist         370000
18  Machine Learning Engineer         323300
19              Data Engineer         310000
20  Machine Learning Engineer         318300
21        

Як виглядає розподіл кількості спеціалістів по країнах проживання?

In [30]:
query = """
SELECT *
FROM df
WHERE
    salary_in_usd >= 300000
    AND year = 2023
    AND company_size = 'M'
"""

result = psql.sqldf(query)
print(result)

    year exp_level emp_type                  job_title  salary salary_curr  \
0   2023        SE       FT   Computer Vision Engineer  342810         USD   
1   2023        SE       FT  Machine Learning Engineer  318300         USD   
2   2023        MI       FT  Machine Learning Engineer  300000         USD   
3   2023        EX       FT               Head of Data  329500         USD   
4   2023        SE       FT  Machine Learning Engineer  304000         USD   
5   2023        EX       FT   Director of Data Science  353200         USD   
6   2023        SE       FT             Data Scientist  317070         USD   
7   2023        SE       FT             Data Architect  376080         USD   
8   2023        MI       FT         Research Scientist  340000         USD   
9   2023        EX       FT              Data Engineer  310000         USD   
10  2023        EX       FT              Data Engineer  310000         USD   
11  2023        SE       FT             Data Scientist  300240  

3. Чи є співробітники, які працювали на Українську компанію повністю віддалено

In [31]:
df.columns

Index(['year', 'exp_level', 'emp_type', 'job_title', 'salary', 'salary_curr',
       'salary_in_usd', 'emp_location', 'remote_ratio', 'company_location',
       'company_size'],
      dtype='object')

In [32]:
query = """
SELECT *
FROM df
WHERE
    company_location = 'UA'
    AND remote_ratio = 100

"""

result = psql.sqldf(query)
print(result)

   year exp_level emp_type       job_title  salary salary_curr  salary_in_usd  \
0  2021        EN       FT  Data Scientist   13400         USD          13400   

  emp_location  remote_ratio company_location company_size  
0           UA           100               UA            L  


4. Вивести всіх співробітників, які в 2023 році працюючи в Німеччині (company_location = 'DE') отримували з/п більшу за $100тис.

In [33]:
query = """
SELECT *
FROM df
WHERE
    company_location = 'DE'
    AND salary_in_usd >= 100000
    AND year = 2023
"""

result = psql.sqldf(query)
print(result)

   year exp_level emp_type                  job_title  salary salary_curr  \
0  2023        SE       FT          Research Engineer  275000         USD   
1  2023        SE       FT          Research Engineer  174000         USD   
2  2023        SE       FT   Deep Learning Researcher  115000         EUR   
3  2023        EN       FT               AI Developer  200000         EUR   
4  2023        SE       FT          Research Engineer  100000         EUR   
5  2023        SE       FT  Machine Learning Engineer  275000         USD   
6  2023        SE       FT  Machine Learning Engineer  174000         USD   

   salary_in_usd emp_location  remote_ratio company_location company_size  
0         275000           DE             0               DE            M  
1         174000           DE             0               DE            M  
2         123405           DE             0               DE            L  
3         214618           DE           100               DE            L  
4  

5. Доопрацювати попередній запит: Вивести з результатів тільки ТОП 5 співробітників за рівнем з/п.

In [34]:
query = """
SELECT *
FROM df
WHERE
    company_location = 'DE'
    AND salary_in_usd >= 100000
    AND year = 2023
ORDER BY salary_in_usd DESC
LIMIT 5
"""

result = psql.sqldf(query)
print(result)

   year exp_level emp_type                  job_title  salary salary_curr  \
0  2023        SE       FT          Research Engineer  275000         USD   
1  2023        SE       FT  Machine Learning Engineer  275000         USD   
2  2023        EN       FT               AI Developer  200000         EUR   
3  2023        SE       FT          Research Engineer  174000         USD   
4  2023        SE       FT  Machine Learning Engineer  174000         USD   

   salary_in_usd emp_location  remote_ratio company_location company_size  
0         275000           DE             0               DE            M  
1         275000           DE             0               DE            M  
2         214618           DE           100               DE            L  
3         174000           DE             0               DE            M  
4         174000           DE             0               DE            M  



6. Додати в попередню таблицю окрім спеціалістів з Німеччини спеціалістів з Канади (CA).

In [35]:
query = """
SELECT *
FROM df
WHERE
    company_location IN ('DE','CA')
    AND salary_in_usd >= 100000
    AND year = 2023
ORDER BY salary_in_usd DESC
LIMIT 5
"""

result = psql.sqldf(query)
print(result)

   year exp_level emp_type                  job_title  salary salary_curr  \
0  2023        SE       FT          Research Engineer  275000         USD   
1  2023        SE       FT  Machine Learning Engineer  275000         USD   
2  2023        SE       FT  Machine Learning Engineer  269000         USD   
3  2023        SE       FT                ML Engineer  260000         USD   
4  2023        SE       FT             Data Scientist  258000         USD   

   salary_in_usd emp_location  remote_ratio company_location company_size  
0         275000           DE             0               DE            M  
1         275000           DE             0               DE            M  
2         269000           CA           100               CA            M  
3         260000           CA           100               CA            M  
4         258000           CA             0               CA            M  


7. Надати перелік країн, в яких в 2021 році спеціалісти "ML Engineer" та "Data Scientist" отримувати з/п в діапазоні між $50тис і $100тис.

In [36]:
query = """
SELECT  DISTINCT company_location
FROM df
WHERE
    job_title IN ('ML Engineer','Data Scientist')
    AND salary_in_usd BETWEEN 50000 AND 100000
    AND year = 2021
"""

result = psql.sqldf(query)
print(result)

  company_location
0               IN
1               US
2               JP
3               CA
4               NG
5               FR
6               DE
7               AT
8               GB


8. Порахувати кількість спеціалістів, які працюючи в середніх компаніях (company_size = M) та в великих компаніях
(company_size = L) працювали віддалено  (remote_ratio=100 або remote_ratio=50).

In [37]:
query = """
SELECT  DISTINCT company_location
FROM df
WHERE
    job_title IN ('ML Engineer','Data Scientist')
    AND salary_in_usd BETWEEN 50000 AND 100000
    AND year = 2021
"""

result = psql.sqldf(query)
print(result)

  company_location
0               IN
1               US
2               JP
3               CA
4               NG
5               FR
6               DE
7               AT
8               GB


9. Вивести кількість країн, які починаються на "С".

In [38]:
query = """
SELECT COUNT(DISTINCT company_location)
FROM df
WHERE
    company_location LIKE ("C%")
"""

result = psql.sqldf(query)
print(result)

   COUNT(DISTINCT company_location)
0                                 8


10. Вивести професії, назва яких не складається з трьох слів.

In [39]:
query = """
SELECT DISTINCT job_title
FROM df
WHERE LENGTH (TRIM(job_title)) - LENGTH(TRIM(REPLACE(job_title,' ',''))) != 2
"""

result = psql.sqldf(query)
print(result)

                                   job_title
0                                ML Engineer
1                             Data Scientist
2                          Applied Scientist
3                               Data Analyst
4                               Data Modeler
5                          Research Engineer
6                         Analytics Engineer
7                            Data Strategist
8                              Data Engineer
9                             Data Architect
10         Applied Machine Learning Engineer
11                              AI Developer
12                        Research Scientist
13                              ETL Engineer
14                              Data Manager
15                           Data Specialist
16                  Director of Data Science
17                            MLOps Engineer
18                              AI Scientist
19        Applied Machine Learning Scientist
20                             AI Programmer
21        

# **Lessons 8 - Групування та агрегація даних в SQL з використанням GROUP BY**

Для кожної професії та відповідного рівня досвіду навести:

- кількість в таблиці
- середню заробітну плату

In [40]:
query = """
SELECT
      job_title
      , exp_level
      , COUNT(*) AS job_nmb
      , ROUND(AVG (salary_in_usd),2) AS salary
FROM df
--WHERE year = 2023
GROUP BY job_title
      , exp_level
ORDER BY 1, 2;
"""

result = psql.sqldf(query)
print(result)

                         job_title exp_level  job_nmb     salary
0    3D Computer Vision Researcher        EN        2   35000.00
1    3D Computer Vision Researcher        MI        1    5409.00
2    3D Computer Vision Researcher        SE        1   10000.00
3                     AI Developer        EN        6  130884.50
4                     AI Developer        MI        2  137510.00
..                             ...       ...      ...        ...
187             Research Scientist        SE       49  179892.98
188         Software Data Engineer        MI        1   75020.00
189         Software Data Engineer        SE        1   50000.00
190             Staff Data Analyst        EX        1   15000.00
191           Staff Data Scientist        SE        1  105000.00

[192 rows x 4 columns]


Для професій, зо зустрічаються лише 1 (або 2) раз, навести заробітну плату


In [41]:
query = """
SELECT
      job_title
      , COUNT(*) AS job_nmb
      , ROUND(AVG (salary_in_usd),2) AS salary
FROM df
WHERE year = 2023
GROUP BY job_title
ORDER BY 2, 3
"""

result = psql.sqldf(query)
print(result)

                        job_title  job_nmb     salary
0   Autonomous Vehicle Technician        1    7000.00
1            Product Data Analyst        1   16414.00
2             Data Analytics Lead        1   17509.00
3         Compliance Data Analyst        1   30000.00
4     Machine Learning Researcher        1   50000.00
..                            ...      ...        ...
60             Research Scientist       55  177539.95
61      Machine Learning Engineer      158  167759.80
62                   Data Analyst      307  114097.48
63                 Data Scientist      370  156812.12
64                  Data Engineer      499  151647.29

[65 rows x 3 columns]


Як змінювалась мінімальна заробітна плата за останні 4 роки?

In [42]:
query = """
SELECT
      year
      , MIN(salary_in_usd)
FROM df
GROUP BY year
ORDER BY year DESC
"""

result = psql.sqldf(query)
print(result)

   year  MIN(salary_in_usd)
0  2023                7000
1  2022                5132
2  2021                5409
3  2020                5707


# Lesson 9. ТОП 5 задач SQL для співбесіди - вкладені запити

ЗАДАЧА 1
Вивести всіх спеціалістів, в яких з/п вище середньої в таблиці

In [43]:
query = """
SELECT job_title
      , salary_in_usd
FROM df
WHERE salary_in_usd>
(
      SELECT AVG(salary_in_usd)
      FROM df
      WHERE year = 2023
)
"""

result = psql.sqldf(query)
print(result)

                               job_title  salary_in_usd
0                         Data Scientist         175000
1                      Applied Scientist         222200
2                         Data Scientist         219000
3                      Applied Scientist         213660
4                         Data Scientist         170000
...                                  ...            ...
1505            Director of Data Science         168000
1506  Applied Machine Learning Scientist         423000
1507                     Data Specialist         165000
1508                      Data Scientist         412000
1509            Principal Data Scientist         151000

[1510 rows x 2 columns]


ЗАДАЧА 2
Вивести всіх спеціалістів, які живуть в країнах, де середня з/п вища за середню серед усіх країн.

--1. Пошук середньої з/п

--2. По кожній країні - середня з/п

--3. Порівнюємо, виводимо передік країн

--4. Спеціалісти, що проживають в цих країнах


In [48]:
query = """
SELECT AVG(salary_in_usd)
FROM df
"""
result = psql.sqldf(query)
print(result)

   AVG(salary_in_usd)
0        137570.38988


In [50]:
query = """
SELECT
	company_location
FROM df
WHERE year = 2023
GROUP BY 1
HAVING AVG(salary_in_usd) >
(
	SELECT AVG(salary_in_usd)
	FROM df
	WHERE year = 2023
)
"""

result = psql.sqldf(query)
print(result)

  company_location
0               IL
1               US


In [52]:
query = """
SELECT *
FROM df
WHERE emp_location IN
(
	SELECT
	company_location
FROM df
WHERE year = 2023
GROUP BY 1
HAVING AVG(salary_in_usd) >
(
	SELECT AVG(salary_in_usd)
	FROM df
	WHERE year = 2023
)
)
"""
result = psql.sqldf(query)
print(result)

      year exp_level emp_type                 job_title  salary salary_curr  \
0     2023        MI       CT               ML Engineer   30000         USD   
1     2023        MI       CT               ML Engineer   25500         USD   
2     2023        SE       FT         Applied Scientist  222200         USD   
3     2023        SE       FT         Applied Scientist  136000         USD   
4     2023        SE       FT            Data Scientist  147100         USD   
...    ...       ...      ...                       ...     ...         ...   
3000  2021        SE       FT           Data Specialist  165000         USD   
3001  2020        SE       FT            Data Scientist  412000         USD   
3002  2021        MI       FT  Principal Data Scientist  151000         USD   
3003  2020        EN       FT            Data Scientist  105000         USD   
3004  2020        EN       CT     Business Data Analyst  100000         USD   

      salary_in_usd emp_location  remote_ratio comp

**ЗАДАЧА 3. Знайти мінімальну заробітну плату серед максимальних з/п по країнах**

In [54]:
query = """
SELECT MIN(t.max_salary)
FROM (
		SELECT
			company_location
			, MAX (Salary_in_usd) as max_salary
		FROM df
		WHERE year=2023
		GROUP BY 1
) as t
"""
result = psql.sqldf(query)
print(result)

   MIN(t.max_salary)
0               7000


In [58]:
query = """

SELECT
	company_location
	, MAX (Salary_in_usd) as max_salary
FROM df
WHERE year=2023
GROUP BY 1
ORDER BY 2 ASC
LIMIT 1
"""
result = psql.sqldf(query)
print(result)

  company_location  max_salary
0               GH        7000
